In [2]:
filename="/Users/kristoferpearson/Dev/scripts/python/Landy/find-auto-renew-issues/database.ini"
section="postgresql"

In [3]:
from configparser import ConfigParser

parser = ConfigParser()
parser.read(filename)

# get section, default to postgresql
ret_config = {}
if parser.has_section(section):
    params = parser.items(section)
    for param in params:
        ret_config[param[0]] = param[1]
else:
    raise KeyError(f"Section {section} not found in the {filename} file")

In [6]:
import psycopg2
from datetime import datetime, timedelta

with psycopg2.connect(**ret_config) as conn:
    cur = conn.cursor()
    cur.execute("""SELECT cg.firm_name, cg.total_incurred
                    FROM claims_gaclaims cg 
                    JOIN cpa_icustomer ci ON cg.policy_num = SUBSTRING(ci.policy_num, 4, 7) 
                    WHERE ci.risk_num = %s
                    LIMIT 5;
                """,
                ("ONHI01-1", ))
    account_data = cur.fetchone()
    loss_date = datetime.strptime(account_data[0], '%m/%d/%Y')
    claim_date = datetime.today() - timedelta(days = 1826)
    if loss_date > claim_date and account_data[1] > 0:
        print(f"{loss_date.strftime('%m/%d/%Y')} is after {claim_date.strftime('%m/%d/%Y')} and loss incurred is {account_data[1]}")


('5/13/2024', Decimal('6000.00'))
05/13/2024 is after 09/28/2019 and loss incurred is 6000.00
